In [7]:
cd ..

D:\zhaixu\Thesis_Code


In [4]:
# ART
# # coding=gbk
import os, pickle, random, sys
import argparse
# 添加模块搜索路径到系统环境变量中
module_path = 'D:\zhaixu\Thesis_Code\dl_amc_backdoor_defense_seq'
sys.path.append(module_path)
os.environ['PYTHONPATH'] = module_path


import warnings
import copy
warnings.filterwarnings('ignore')
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from art.estimators.classification import KerasClassifier
from art.attacks.poisoning import PoisoningAttackBackdoor
from art.attacks.poisoning.perturbations import add_pattern_bd, add_single_bd, insert_image
from art.utils import load_mnist, preprocess
from art.defences.detector.poison import ActivationDefence
from art.defences.transformer.poisoning import NeuralCleanse
from art.estimators.poison_mitigation import KerasNeuralCleanse
from sklearn.model_selection import train_test_split

In [5]:
def parse_arguments():
    parser = argparse.ArgumentParser(description='AC PARAMs')

    # 添加命令行参数
    parser.add_argument('--TRIGGER_TYPE', type=str, default='spectrum_shift')
    parser.add_argument('--GPU_NUM', type=str, default='0')
    parser.add_argument('--POS_RATE', type=float, default=0.1)
    parser.add_argument('--MODEL_NAME', type=str, default='CNN2')
    parser.add_argument('--EPOCH', type=int, default=0)

    return parser.parse_args()

def compute_TPR_FPR(y_true, y_pred):
    from sklearn.metrics import confusion_matrix

    # 计算混淆矩阵
    conf_matrix = confusion_matrix(y_true, y_pred)

    # 提取混淆矩阵的各个元素
    TP = conf_matrix[0, 0]  # True Positives
    FN = conf_matrix[0, 1]  # False Negatives
    FP = conf_matrix[1, 0]  # False Positives
    TN = conf_matrix[1, 1]  # True Negatives

    # 计算 True Positive Rate (TPR) 和 False Positive Rate (FPR)
    TPR = TP / (TP + FN)
    FPR = FP / (FP + TN)

    print(f'True Positive Rate (TPR): {TPR:.2f}')
    print(f'False Positive Rate (FPR): {FPR:.2f}')

    return TPR, FPR

def metrics(y_true, y_pred):

    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

    # 计算混淆矩阵
    conf_matrix = confusion_matrix(y_true, y_pred)

    # 计算准确率
    accuracy = accuracy_score(y_true, y_pred)

    # 计算精确度
    precision = precision_score(y_true, y_pred,pos_label=0)

    # 计算召回率
    recall = recall_score(y_true, y_pred,pos_label=0)

    # 计算 F1 分数
    f1 = f1_score(y_true, y_pred,pos_label=0)

    print("Confusion Matrix:")
    print(conf_matrix)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

    # model evaluation
def evaluation(model,X_test, Y_test,test_idx,mods,snrs,lbl):
    classes = mods
    acc = []
    for snr in snrs:

        # extract classes @ SNR
        test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
        test_X_i = X_test[np.where(np.array(test_SNRs) == snr)]
        test_Y_i = Y_test[np.where(np.array(test_SNRs) == snr)]

        # estimate classes
        test_Y_i_hat = model.predict(test_X_i)
        conf = np.zeros([len(classes), len(classes)])

        for i in range(0, test_X_i.shape[0]):
            j = list(test_Y_i[i, :]).index(1)
            k = int(np.argmax(test_Y_i_hat[i, :]))
            conf[j, k] = conf[j, k] + 1

        cor = np.sum(np.diag(conf))
        ncor = np.sum(conf) - cor
        #print("Overall Accuracy: ", cor / (cor+ncor))
        print("snr:",snr,"acc:",cor / (cor + ncor))
        acc.append(1.0 * cor / (cor + ncor))
    acc_mean = sum(acc) / len(acc)
    print('acc_mean: ',acc_mean)
    acc.append(acc_mean)
    return acc

In [8]:
from trigger_config import load_data
from trigger_config import set_trigger_config
X_train,X_test,Y_train,Y_test,mods,lbl,snrs,train_idx,test_idx = load_data()

筛选后的数据集总数： 110000
信噪比范围: 0 到 18
调制方式 11 种: ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM']


In [9]:
X_train_modified, Y_train_modified = set_trigger_config(X_train.copy(), Y_train.copy(), pos_rate=0.1,
                                                        trigger_type='spectrum_shift', data_type='train')

x_train, x_val, y_train, y_val = train_test_split(X_train_modified, Y_train_modified, test_size=0.111, random_state=42)

Poisoned training data.


In [10]:
# load model
from tensorflow.keras.models import load_model
poisoned_model = load_model('D:/zhaixu/Thesis_Code/dl_amc_backdoor/all_to_one/saved_model/'
                            'posioned_CNN2_Spectrum_shift_EPOCH_100_POS_RATE_0.1.h5')

# reshape input
input_shape = poisoned_model.get_input_shape_at(0)
X_train_modified = X_train_modified.reshape((X_train_modified.shape[0],) + tuple(input_shape[1:]))
x_train = x_train.reshape((x_train.shape[0],) + tuple(input_shape[1:]))
x_val = x_val.reshape((x_val.shape[0],) + tuple(input_shape[1:]))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# detect
from art.defences.detector.poison import SpectralSignatureDefense
from art.defences.transformer.poisoning import STRIP
from art.defences.detector.poison import ActivationDefence

In [15]:
classifier = KerasClassifier(model=poisoned_model)

defence = ActivationDefence(classifier, X_train_modified, Y_train_modified)
report, is_clean_lst = defence.detect_poison(nb_clusters=2, nb_dims=11, reduce="PCA")

In [17]:
# 获取第7个位置的值
index = 7
y_true = np.array([0 if sample[index] == 1 else 1 for sample in Y_train_modified])

y_pred = is_clean_lst
y_pred = np.array(is_clean_lst)
compute_TPR_FPR(y_true,y_pred)

True Positive Rate (TPR): 0.00
False Positive Rate (FPR): 0.40


(0.00033444816053511704, 0.39915583694324114)

In [14]:
metrics(y_true, y_pred)

Confusion Matrix:
[[ 1869 14177]
 [30209 41756]]
Accuracy: 0.495676676779039
Precision: 0.05826423093709084
Recall: 0.1164776268228842
F1 Score: 0.07767434128501371


In [45]:
# retrain model
# filter data
is_clean_lst = np.array(is_clean_lst)
clean_indices = np.where(is_clean_lst == 1)[0]
cleaned_x = x_train[clean_indices]
cleaned_y = y_train[clean_indices]

In [31]:
from mltools import build_model, train
model = build_model(target_model='CNN2')
model, history = train(model, cleaned_x, cleaned_y, x_val, y_val, nb_epoch=10, batch_size=1024)

Train on 55933 samples, validate on 10989 samples
Epoch 1/10
55296/55933 [============================>.] - ETA: 0s - loss: 2.2837 - accuracy: 0.2508
Epoch 00001: val_loss improved from inf to 2.33396, saving model to VGG_dr0.5.h5
55933/55933 [==============================] - 20s 359us/sample - loss: 2.2826 - accuracy: 0.2509 - val_loss: 2.3340 - val_accuracy: 0.1742
Epoch 2/10
55296/55933 [============================>.] - ETA: 0s - loss: 1.9452 - accuracy: 0.3214
Epoch 00002: val_loss did not improve from 2.33396
55933/55933 [==============================] - 15s 263us/sample - loss: 1.9420 - accuracy: 0.3215 - val_loss: 2.6000 - val_accuracy: 0.2834
Epoch 3/10
55296/55933 [============================>.] - ETA: 0s - loss: 1.4006 - accuracy: 0.4387
Epoch 00003: val_loss did not improve from 2.33396
55933/55933 [==============================] - 15s 269us/sample - loss: 1.3991 - accuracy: 0.4388 - val_loss: 2.8672 - val_accuracy: 0.3924
Epoch 4/10
55296/55933 [=======================

In [51]:
# no defense
#args = parse_arguments()
print('No defense eval\n')
X_test_benign = X_test.reshape((X_test.shape[0],) + tuple(input_shape[1:]))

X_test_modified, Y_test_modified = set_trigger_config(X_test.copy(), Y_test.copy(), pos_rate=0.1,
                                                        trigger_type='spectrum_shift', data_type='test')

X_test_modified = X_test_modified.reshape((X_test_modified.shape[0],) + tuple(input_shape[1:]))

print('CA')
acc_ca_no_defense = evaluation(poisoned_model,X_test_benign,Y_test,test_idx,mods,snrs,lbl)
print('ASR')
acc_asr_no_defense = evaluation(poisoned_model,X_test_modified,Y_test_modified,test_idx,mods,snrs,lbl)

No defense eval

Evaluate ASR on poisoned test data.
CA
snr: 0 acc: 0.8255481410867492
snr: 2 acc: 0.8473782771535581
snr: 4 acc: 0.8651982378854626
snr: 6 acc: 0.8686868686868687
snr: 8 acc: 0.8597402597402597
snr: 10 acc: 0.837568058076225
snr: 12 acc: 0.8494138863841298
snr: 14 acc: 0.8618481244281794
snr: 16 acc: 0.8550323176361958
snr: 18 acc: 0.8880931065353626
acc_mean:  0.8558507277612991
ASR
snr: 0 acc: 0.8751191611058151
snr: 2 acc: 0.8820224719101124
snr: 4 acc: 0.9118942731277533
snr: 6 acc: 0.9182736455463728
snr: 8 acc: 0.9428571428571428
snr: 10 acc: 0.941923774954628
snr: 12 acc: 0.957619477006312
snr: 14 acc: 0.9478499542543458
snr: 16 acc: 0.9529085872576177
snr: 18 acc: 0.9409131602506714
acc_mean:  0.9271381648270772


In [53]:
# eval repaired model
print('Repaired model eval\n')
print('CA')
acc_ca_repaired = evaluation(model,X_test_benign,Y_test,test_idx,mods,snrs,lbl)
print('ASR')
acc_asr_repaired = evaluation(model,X_test_modified,Y_test_modified,test_idx,mods,snrs,lbl)

Repaired model eval

CA
snr: 0 acc: 0.5853193517635844
snr: 2 acc: 0.6554307116104869
snr: 4 acc: 0.6599118942731278
snr: 6 acc: 0.6519742883379247
snr: 8 acc: 0.6597402597402597
snr: 10 acc: 0.631578947368421
snr: 12 acc: 0.6600541027953111
snr: 14 acc: 0.67612076852699
snr: 16 acc: 0.6638965835641736
snr: 18 acc: 0.6812891674127126
acc_mean:  0.6525316075392992
ASR
snr: 0 acc: 0.8179218303145853
snr: 2 acc: 0.8820224719101124
snr: 4 acc: 0.9145374449339208
snr: 6 acc: 0.9274563820018366
snr: 8 acc: 0.9307359307359307
snr: 10 acc: 0.9319419237749547
snr: 12 acc: 0.9386834986474302
snr: 14 acc: 0.9341262580054894
snr: 16 acc: 0.9519852262234534
snr: 18 acc: 0.9355416293643688
acc_mean:  0.9164952595912081
